In [8]:
import numpy as np
import keras
from keras import Sequential
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Concatenate, Flatten
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

np.random.seed(42)

X_shape = (20, 8) # each X consists of 20 samples of 8 features
y_shape = (1,) # the result is 1 bit
event_num = 2048

"""
keras Batch Generator that imitates a batchgenerator, which reads batches from a database
"""
class BatchGen(keras.utils.Sequence):
  def __init__(self, batch_size=16):
    self.batch_size=batch_size
    self.data_length = event_num # event_num (X, y) pairs in our fictional database
    
    # Generate random data
    self.X = np.random.randint(0,2,size=(self.data_length, *X_shape))
    self.y = np.random.randint(0,2,size=(self.data_length, *y_shape))
    
  def __len__(self):
    return int(np.ceil(self.data_length/float(self.batch_size)))
  
  def __getitem__(self, index):
    slc = slice(index*self.batch_size, (index+1)*self.batch_size)
    return (self.X[slc], self.y[slc])

"""
Simple keras model to use our batch generator on something
"""  
class SimpleDecoder:
    def __init__(self, xshape, hidden_size=64):
        self.hidden_size=hidden_size
        self.xshape=xshape
        pass
    
    def create_model(self):
        # This returns a tensor
        input_syndr = Input(shape=(self.xshape))
        
        x = LSTM(self.hidden_size, return_sequences=True)(input_syndr)
        x = LSTM(self.hidden_size, return_sequences=True)(x)
        x = Flatten()(x)
        x = Dense(self.hidden_size, activation='relu')(x)
        predictions = Dense(1, activation='softmax')(x)
        
        # optimizer
        sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
        
        model = Model(inputs=input_syndr, outputs=predictions)
        model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
        return model


model=SimpleDecoder(xshape=X_shape).create_model()

bg=BatchGen()
model.fit_generator(generator=bg,
                    epochs=4,
                    validation_data=bg,
                    use_multiprocessing=True,
                    workers=1)

Epoch 1/4
128/128 [==============================] - 8s 61ms/step - loss: 7.5975 - acc: 0.5234 - val_loss: 7.5975 - val_acc: 0.5234
Epoch 2/4
128/128 [==============================] - 6s 47ms/step - loss: 7.5975 - acc: 0.5234 - val_loss: 7.5975 - val_acc: 0.5234
Epoch 3/4
128/128 [==============================] - 6s 47ms/step - loss: 7.5975 - acc: 0.5234 - val_loss: 7.5975 - val_acc: 0.5234
Epoch 4/4
128/128 [==============================] - 6s 47ms/step - loss: 7.5975 - acc: 0.5234 - val_loss: 7.5975 - val_acc: 0.5234


In [5]:
import hashlib
hashlib.md5(b"valami").hexdigest()

'249d6c99eff6d0ef6c470e4254629323'